# Nystrom_common class

*     Note: k_approx to be added in

In [ ]:
# If using colab, make sure you install via:  !pip install pykeops[full] > log.log
# makes plot outputs appear and be stored within the notebook
%matplotlib inline 

!apt-get install cuda=10.2.89-1
!pip install si_prefix
!pip install sphinx
!pip install pykeops[full] > log.log

In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
cd /content/drive/MyDrive/keops

/content/drive/MyDrive/keops


In [46]:
import sys
sys.path.append('/content/drive/MyDrive/keops')

In [47]:
from numpy_utils import numpytools
from torch_utils import torchtools

# from common.nystrom import Nystrom_common -> needed when integrating into the lib

In [48]:
import numpy as np
import pykeops
import numbers
import torch

# For LinearOperator math
from scipy.sparse.linalg import aslinearoperator, eigsh
from scipy.sparse.linalg.interface import IdentityOperator

## Abstract class

In [49]:
class Nystrom_common:

    def __init__(self, n_components=100, kernel='rbf', sigma:float = 1.,
            exp_sigma:float = 1.0, eps:float = 0.05, mask_radius:float = None,
            k_means = 10, n_iter:int = 10, inv_eps:float = None, dtype = np.float32, 
            backend = None, verbose = False, random_state=None, tools = None): 

        self.n_components = n_components
        self.kernel = kernel
        self.random_state = random_state
        self.sigma = sigma
        self.exp_sigma = exp_sigma
        self.eps = eps
        self.mask_radius = mask_radius
        self.k_means = k_means
        self.n_iter = n_iter
        self.dtype = dtype
        self.tools = None
        self.verbose = verbose # -> NUMPY ONLY

        ### THIS IS DIFFERENT -> CAN IT BE MERGED OR NOT?
        if not backend:
            self.backend = 'GPU' if pykeops.config.gpu_available else 'CPU'
        else:
            self.backend = backend
        # --------end of code to check

        # as of now torch has some extra lines for linear kernel -> delete that
        if inv_eps:
            self.inv_eps = inv_eps
        else:
            self.inv_eps = 1e-8

        if not mask_radius:
            if kernel == 'rbf':
                self.mask_radius = 2* np.sqrt(2) * self.sigma
            elif kernel == 'exp':
                self.mask_radius = 8 * self.exp_sigma

    def fit(self, x):
        ### UPDATE DOCSTRING
        ''' 
        Args:   x = numpy array of shape (n_samples, n_features)
        Returns: Fitted instance of the class
        '''

        # specific to numpy, but i set verbose = None in torch so this should be 
        # skipped
        if self.verbose:
            print(f'Working with backend = {self.backend}')
        
        # Basic checks

        ### this uses a common method - how do i personalize the error message?
        assert self.tools.is_tensor(x), 'Input to fit(.) must be an array.'

        ### can i use .shape for a tensor, too ?
        assert x.shape[0] >= self.n_components, 'The application needs X.shape[0] >= n_components.'

        assert self.exp_sigma > 0, 'Should be working with decaying exponential.'

        # Update dtype
        self._update_dtype(x) ### COMMENTED OUT FOR TORCH

        # Number of samples
        n_samples = x.shape[0]

        # Define basis
        rnd = self._check_random_state(self.random_state)
        inds = rnd.permutation(n_samples) 
        basis_inds = inds[:self.n_components] 
        basis = x[basis_inds]
        # Build smaller kernel

        basis_kernel = self._pairwise_kernels(basis, dense=False)

        # Decomposition is an abstract method that needs to be defined in each class
        self.normalization_ = self._decomposition_and_norm(basis_kernel)

        self.components_ = basis
        self.component_indices_ = inds

        return self

    def _decomposition_and_norm(self, basis_kernel):
        """
        to be defined in the lang-specific classes
        """
        pass

    def transform(self, x):
        ### UPDATE DOCSTRING
        ''' Applies transform on the data.
        
        Args:
            X [np.array] = data to transform
        Returns
            X [np.array] = data after transformation
        '''

        K_nq = self._pairwise_kernels(x, self.components_, dense=True)
        x_new = K_nq @ self.tools.transpose(self.normalization_) ### COMMON TRANSFORM NEEDED
        return x_new

    def _pairwise_kernels(self, x, y = None, dense = False):
        ### UPDATE DOCSTRING
        '''Helper function to build kernel
        
        Args:   x[np.array] = data
                y[np.array] = array 
                dense[bool] = False to work with lazy tensor reduction,
                              True to work with dense arrays
        Returns:
                K_ij[LazyTensor] if dense = False
                K_ij[np.array] if dense = True

        '''
        if y is None:
            y = x
        if self.kernel == 'rbf':
            x /= self.sigma
            y /= self.sigma
            if dense:
                x_i, x_j = x[:, None, :], y[None, :, :]
                K_ij = self.tools.exp( -(( (x_i - x_j)**2 ).sum(axis=2)) )
            else:
                x_i, x_j = LazyTensor(x[:, None, :]), LazyTensor(y[None, :, :])
                K_ij = ( -(( (x_i - x_j)**2 ).sum(dim=2) ) ).exp()
                # block-sparse reduction preprocess
                K_ij = self._Gauss_block_sparse_pre(x, y, K_ij)
        elif self.kernel == 'exp':
            x /= self.exp_sigma
            y /= self.exp_sigma
            if dense:
                x_i, x_j = x[:, None, :], y[None, :, :]
                K_ij =  self.tools.exp(-self.tools.sqrt( ( ((x_i - x_j) ** 2).sum(axis=2) )))
            else:
                x_i, x_j = LazyTensor(x[:, None, :]), LazyTensor(y[None, :, :])
                K_ij = (-(((x_i - x_j) ** 2).sum(-1)).sqrt()).exp()
                # block-sparse reduction preprocess
                K_ij = self._Gauss_block_sparse_pre(x, y, K_ij) # TODO 
       
        if not dense:
            K_ij.backend = self.backend
        
        return K_ij

    def _Gauss_block_sparse_pre(self, x, y, K_ij):
        ''' 
        Helper function to preprocess data for block-sparse reduction
        of the Gaussian kernel
    
        Args: 
            x[np.array], y[np.array] = arrays giving rise to Gaussian kernel K(x,y)
            K_ij[LazyTensor_n] = symbolic representation of K(x,y)
            eps[float] = size for square bins
        Returns:
            K_ij[LazyTensor_n] = symbolic representation of K(x,y) with 
                                set sparse ranges
        '''
        # labels for low dimensions
        if x.shape[1] < 4 or y.shape[1] < 4:
            x_labels = grid_cluster(x, self.eps) 
            y_labels = grid_cluster(y, self.eps) 
            # range and centroid per class
            x_ranges, x_centroids, _ = cluster_ranges_centroids(x, x_labels)
            y_ranges, y_centroids, _ = cluster_ranges_centroids(y, y_labels)
        else:
        # labels for higher dimensions
            x_labels, x_centroids = self._KMeans(x)
            y_labels, y_centroids = self._KMeans(y)
            # compute ranges
            x_ranges = cluster_ranges(x_labels)
            y_ranges = cluster_ranges(y_labels)

        # sort points
        x, x_labels = sort_clusters(x, x_labels)
        y, y_labels = sort_clusters(y, y_labels) 
        # Compute a coarse Boolean mask:
        if self.kernel == 'rbf':
            D = self.tools.arraysum((x_centroids[:, None, :] - y_centroids[None, :, :]) ** 2, 2)
        elif self.kernel == 'exp':
            D = self.tools.sqrt(self.tools.arraysum((x_centroids[:, None, :] - y_centroids[None, :, :]) ** 2, 2))
        keep = D < (self.mask_radius) ** 2
        # mask -> set of integer tensors
        ranges_ij = from_matrix(x_ranges, y_ranges, keep)
        K_ij.ranges = ranges_ij  # block-sparsity pattern

        return K_ij

    def _KMeans(self,x):
        ''' KMeans with Pykeops to do binning of original data.
        Args:
            x[np.array] = data
            k_means[int] = number of bins to build
            n_iter[int] = number iterations of KMeans loop
        Returns:
            labels[np.array] = class labels for each point in x
            clusters[np.array] = coordinates for each centroid
        '''
        N, D = x.shape  
        clusters = self.tools.copy(x[:self.k_means, :])  # initialization of clusters
        x_i = LazyTensor(x[:, None, :])  

        for i in range(self.n_iter):

            clusters_j = LazyTensor(clusters[None, :, :])  
            D_ij = ((x_i - clusters_j) ** 2).sum(-1)  # points-clusters kernel
            labels = self._astype(D_ij.argmin(axis=1), int).reshape(N)  # Points -> Nearest cluster
            Ncl = self._astype(self.tools.bincount(labels), self.dtype)  # Class weights
            for d in range(D):  # Compute the cluster centroids with bincount:
                clusters[:, d] = self.tools.bincount(labels, weights=x[:, d]) / Ncl

        return labels, clusters

    def _astype(self, data, type):
        return data

    def _update_dtype(self,x):
        ''' Helper function that sets dtype to that of 
            the given data in the fitting step.
            
        Args:
            x [np.array] = raw data to remap
        Returns:
            nothing
        '''
        self.dtype = x.dtype
        self.inv_eps = np.array([self.inv_eps]).astype(self.dtype)[0]

    def _check_random_state(self, seed):
        '''Set/get np.random.RandomState instance for permutation

        Args
            seed[None, int] 
        Returns:
            numpy random state
        '''
        if seed is None:
            return np.random.mtrand._rand
        elif type(seed) == int:
            return np.random.RandomState(seed)
        raise ValueError(f'Seed {seed} must be None or an integer.')



### Data

In [50]:
from sklearn.datasets import make_blobs

# data

length = 10000
n_sampling = 500

# data set with clusters
data_clustered, _ = make_blobs(n_samples= length, n_features=3, centers=5)
data_clustered_t = torch.tensor(data_clustered, dtype=torch.float32)

## Numpy class

In [51]:
from pykeops.numpy import LazyTensor
from pykeops.numpy.cluster import grid_cluster
from pykeops.numpy.cluster import from_matrix
from pykeops.numpy.cluster import cluster_ranges_centroids, cluster_ranges
from pykeops.numpy.cluster import sort_clusters

class Nystrom_NK(Nystrom_common):

    def __init__(self, n_components=100, kernel='rbf', sigma:float = 1.,
            exp_sigma:float = 1.0, eps:float = 0.05, mask_radius:float = None,
            k_means = 10, n_iter:int = 10, inv_eps:float = None, dtype = np.float32, 
            backend = None, verbose = False, random_state=None, tools = None):

        super().__init__(n_components, kernel, sigma, exp_sigma, eps, mask_radius,
                         k_means, n_iter, inv_eps, dtype, backend, verbose, random_state)
        
        self.tools = numpytools
        
    def _decomposition_and_norm(self, basis_kernel):

        K_linear = aslinearoperator(basis_kernel)
        # K <- K + eps
        K_linear = K_linear + IdentityOperator(K_linear.shape, dtype=self.dtype) * self.inv_eps
        k = K_linear.shape[0] - 1
        S, U = eigsh(K_linear, k=k, which='LM')
        S = np.maximum(S, 1e-12)

        return np.dot(U / np.sqrt(S), U.T)

    def K_approx(self, x:np.array) -> np.array:
        ''' Function to return Nystrom approximation to the kernel.
        
        Args:
            X[np.array] = data used in fit(.) function.
        Returns
            K[np.array] = Nystrom approximation to kernel'''
    
        K_nq = self._pairwise_kernels(x, self.components_, dense=True)
        # For arrays: K_approx = K_nq @ K_q_inv @ K_nq.T
        # But to use @ with lazy tensors we have:
        K_q_inv = self.normalization_.T @ self.normalization_
        K_approx = K_nq @ (K_nq @ K_q_inv ).T
        return K_approx.T 

    def _astype(self, data, d_type):
        return data.astype(d_type)


In [52]:
Nystrom_n = Nystrom_NK(n_components = n_sampling, kernel = 'rbf', random_state = 0).fit(data_clustered)
X_new = Nystrom_n.transform(data_clustered)
print(X_new)

[[-2.42059824e-13 -1.55609038e-12  5.54878433e-13 ... -1.32998794e-12
  -4.34449577e-13  8.81071264e-02]
 [ 4.14120202e-12 -1.18383725e-11  8.37015162e-12 ...  3.33310623e-12
  -1.87731258e-11  3.43142639e-04]
 [-8.77232119e-12 -1.44424937e-11  7.79232872e-13 ...  2.95213893e-11
   2.40884278e+02 -8.81582458e-12]
 ...
 [-6.23601415e-12  5.31719945e-12 -1.08875306e-11 ...  9.50387839e-12
   2.57495022e-11  1.18679847e-01]
 [-1.10843734e-12 -1.18425358e-12  9.16791026e-13 ... -2.55597858e-12
   4.18685743e-13  1.11279659e-01]
 [-4.53859328e-13 -4.25322301e-13 -3.54006946e-13 ...  1.46565331e-12
   2.42316418e-12 -2.22560924e-05]]


## Torch class

In [53]:
from pykeops.torch import LazyTensor
from pykeops.torch.cluster import grid_cluster
from pykeops.torch.cluster import from_matrix
from pykeops.torch.cluster import cluster_ranges_centroids, cluster_ranges
from pykeops.torch.cluster import sort_clusters

class Nystrom_TK(Nystrom_common):
    
    def __init__(self, n_components=100, kernel='rbf', sigma:float = 1.,
            exp_sigma:float = 1.0, eps:float = 0.05, mask_radius:float = None,
            k_means = 10, n_iter:int = 10, inv_eps:float = None, dtype = np.float32, 
            backend = None, verbose = False, random_state=None, tools = None):
        
        super().__init__(n_components, kernel, sigma, exp_sigma, eps, mask_radius,
                         k_means, n_iter, inv_eps, dtype, backend, verbose, random_state)
        
        self.tools = torchtools

        # as of now torch doesn't deal w/ these two, so just make them null?
        self.backend = None
        self.verbose = None

    # is this enough to overwrite the superclass function & make it do nothing?
    def _update_dtype(self, x):
        pass
    
    def _decomposition_and_norm(self, basis_kernel):
        
        basis_kernel = basis_kernel @ torch.diag(torch.ones(basis_kernel.shape[1]))
        U, S, V = torch.svd(basis_kernel)
        S = torch.maximum(S, torch.ones(S.size()) * 1e-12)

        return torch.mm(U / np.sqrt(S), V.t())
    
    def K_approx(self, X: torch.tensor) -> torch.tensor:
        ''' Function to return Nystrom approximation to the kernel.
        Args:
            X[torch.tensor] = data used in fit(.) function.
        Returns
            K[torch.tensor] = Nystrom approximation to kernel'''

        K_nq = self._pairwise_kernels(X, self.components_)
        K_approx = K_nq @ self.normalization_ @ K_nq.t()
        return K_approx

In [55]:
Nystrom_t = Nystrom_TK(n_components = n_sampling, kernel = 'rbf', random_state = 0).fit(data_clustered_t)
X_new = Nystrom_n.transform(data_clustered_t)
print(X_new)

tensor([[-2.4206e-13, -1.5561e-12,  5.5488e-13,  ..., -1.3300e-12,
         -4.3445e-13,  8.8107e-02],
        [ 4.1412e-12, -1.1838e-11,  8.3701e-12,  ...,  3.3331e-12,
         -1.8773e-11,  3.4314e-04],
        [-8.7723e-12, -1.4442e-11,  7.7925e-13,  ...,  2.9521e-11,
          2.4088e+02, -8.8158e-12],
        ...,
        [-6.2360e-12,  5.3172e-12, -1.0888e-11,  ...,  9.5039e-12,
          2.5750e-11,  1.1868e-01],
        [-1.1084e-12, -1.1843e-12,  9.1679e-13,  ..., -2.5560e-12,
          4.1869e-13,  1.1128e-01],
        [-4.5386e-13, -4.2532e-13, -3.5401e-13,  ...,  1.4657e-12,
          2.4232e-12, -2.2256e-05]], dtype=torch.float64)
